In [1]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta


%matplotlib inline  

In [2]:
api_key = '00ef9804fcde0edd93b1b4821ee2f06a'
# api_key = 'c81352430e2fe3c941faf0814227562b'

data

In [3]:
''' Колонки '''

ticker = 'AI'

# tickers_list = ['AXP', 'INTC']
# 








required_year = '2022'

''' время '''
stime='2015-01-01'
period = '1mo'  # '1d' '1wk'

In [4]:
def change_percent_all_columns(df):
    x = df.columns.to_list()
    for i in range(len(x)):
        v = df[x[i]].to_list()
        l = []
        for y in range(len(v)):
            try:
                if y != len(v):
                    if v[y] < 0 and v[y+1] > 0:
                        l.append('Minus')
                    elif v[y] > 0 and v[y+1] < 0:
                        l.append('Plus')
                    else:
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv) 
                else:
                    vv = 0
                    l.append(vv)
                    # print(vv)
            except:
                l.append(0)
        df[f'change % {x[i]}'] = l
    return df

In [5]:
def one_stock_analyse(ticker, api_key, required_year):

    ''' запрос к yahoo за ценами '''
    def get_data_from_ticker(tick, start_time='2017-01-01', interval='1mo'):
        ticker = yf.Ticker(tick)

        df = ticker.history(start=start_time, end=None, interval=interval)
        x = pd.DataFrame(df)
        x.rename(columns={"Close": tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        return z

    df = get_data_from_ticker(ticker)
    
    ''' получаем цену на период отчета '''
    income_statement = fa.income_statement(ticker, api_key)
    accepted_date = income_statement.T['acceptedDate'][0][:10]
    df_report = get_data_from_ticker(ticker, accepted_date)
    report_price = df_report.iloc[0,0]
    
    ''' получаем актуальную цену акции '''
    today = datetime.date.today()
    sdate = today - relativedelta(months=3)
    sdate = str(sdate)
    df_current_price = get_data_from_ticker(ticker, sdate, '1d')
    current_price = df_current_price.iloc[-1,0]
    
    ''' меняем индекс для корреляции с df ключевых метрик '''
    def change_index(df):
        df_index = df.index.to_list()
        for i in range(len(df_index)):
            t = df_index[i].date()
            df_index[i] = t.strftime('%Y-%m-%d')
        df['Date_index'] = df_index
        df = df.set_index('Date_index')
        return df

    df = change_index(df)


    def get_key_metricks(ticker, api_key, required_year):
        key_metrics = fa.key_metrics(ticker, api_key, period='annual')
        df_key_metr = key_metrics.T
        df_key_metr = df_key_metr.loc[required_year:'2000'].copy(deep=True)
        return df_key_metr

    
    
    df_key_metr = get_key_metricks(ticker, api_key, required_year)
    
    
    if df_key_metr.index[0] == required_year and f'{int(required_year)+1}-01-01' in df.index.to_list():
        last_year = True
    else:
        last_year = False



    def get_index(df):
        index_years = df.index.to_list()
        return index_years

    index_years = get_index(df_key_metr)



    def get_price_from_df(df, list_dates):
        new_list = []
        for i in range(len(list_dates)):
            x = int(list_dates[i])
            x += 1
            x = str(x)
            try:
                r = df.loc[f'{x}-01-01']
                r = float(r)
                new_list.append(r)
            except:
                new_list.append(0)
                
        return new_list

    stock_year_prices = get_price_from_df(df, index_years)


    df_key_metr[ticker] = list(stock_year_prices)
    df_key_metr.drop('period', axis=1, inplace=True)

    def change_percent_all_columns(df):
        x = df.columns.to_list()
        for i in range(len(x)):
            v = df[x[i]].to_list()
            l = []
            for y in range(len(v)):
                try:
                    if y != len(v):
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv)
                    else:
                        vv = 0
                        l.append(vv)
                except:
                    l.append(0)
            df[f'change % {x[i]}'] = l
        return df
    
    df_with_changes = change_percent_all_columns(df_key_metr)
    
    # if df_with_changes[f'change % {ticker}'][0] > df_with_changes[f'change % {ticker}'][1] and df_with_changes['change % debtToAssets'][0] < df_with_changes['change % debtToAssets'][1]:
    #     better_less_last = "Short"
    if df_with_changes[f'change % {ticker}'][0] < df_with_changes[f'change % {ticker}'][1] and df_with_changes['change % debtToAssets'][0] > df_with_changes['change % debtToAssets'][1] and df_with_changes['change % debtToAssets'][0] > df_with_changes[f'change % {ticker}'][0]:
        better_less_last = "Long"
    else:
        better_less_last = False
   

    
    change_price = current_price / (report_price/100) - 100
    
    
    if last_year == True and better_less_last != False:
        # df_last = df_with_changes.reindex(index=df_with_changes.index[::-1])
        r = [ticker, accepted_date, report_price, current_price, change_price, df_with_changes['change % debtToAssets'][0], better_less_last]
    else:
        r = None 
        
        
    print(r)
    return r
    # return df_report
    # return accepted_date

results of stock's data

In [6]:
def get_results_da_price(tickers_list, api_key, required_year):
    l = []
    for i in range(len(tickers_list)):
        r = one_stock_analyse(tickers_list[i], api_key, required_year)
        if r == None:
            pass
        else:
            l.append(r)
    return l
  
df = pd.DataFrame(get_results_da_price(tickers_list, api_key, required_year), columns = ['Ticker', 'Accepted_Date', 'Price', 'Current Price', 'Change % price', 'change % debtToAssets', 'Side'])
df

['RH', '2022-03-30', 336.1199951171875, 318.42999267578125, -5.263002111861468, -6.737912479552264, 'Long']
None
None
None
['IRBT', '2022-02-15', 63.400001525878906, 42.310001373291016, -33.26498366719956, 21.932462794313622, 'Long']
['AOUT', '2022-07-14', 8.59000015258789, 10.680000305175781, 24.330618340655576, 49.753536861643056, 'Long']
None
None
None
None
None
None
None
None
None
None
['LOW', '2022-03-21', 193.76382446289062, 215.8699951171875, 11.40882242367725, 14.303103256792042, 'Long']
None
None
None
None


,Ticker,Accepted_Date,Price,Current Price,Change % price,change % debtToAssets,Side
0,RH,2022-03-30,336.119995,318.429993,-5.263002,-6.737912,Long
1,IRBT,2022-02-15,63.400002,42.310001,-33.264984,21.932463,Long
2,AOUT,2022-07-14,8.590000,10.680000,24.330618,49.753537,Long
3,LOW,2022-03-21,193.763824,215.869995,11.408822,14.303103,Long


results

In [7]:
l  = []
for i in range(len(df['Change % price'])):
    if df['Side'][i] == 'Long':
        r = df['Change % price'][i] - df['change % debtToAssets'][i]
        r = r *(-1) if r < 0 else r
        l.append(r)
    elif df['Side'][i] == 'Short':
        r = df['Change % price'][i] - df['change % debtToAssets'][i]
        r = r *(-1) if r > 0 else r

        l.append(r)
    else:
        l.append(0)

df['Differense'] = l

In [8]:
df

,Ticker,Accepted_Date,Price,Current Price,Change % price,change % debtToAssets,Side,Differense
0,RH,2022-03-30,336.119995,318.429993,-5.263002,-6.737912,Long,1.474910
1,IRBT,2022-02-15,63.400002,42.310001,-33.264984,21.932463,Long,55.197446
2,AOUT,2022-07-14,8.590000,10.680000,24.330618,49.753537,Long,25.422919
3,LOW,2022-03-21,193.763824,215.869995,11.408822,14.303103,Long,2.894281


add_key_metrics

In [9]:

# def add_key_metrics(df):
#     v = 'revenuePerShare'
#     v1 = 'netIncomePerShare'
#     v2 = 'inventoryTurnover'
#     v3 = 'capexPerShare'
#     v4 = 'cashPerShare'
#     v5 = 'bookValuePerShare'
#     v6 = 'daysPayablesOutstanding'
#     v7 = 'receivablesTurnover'
#     l = []
#     l1 = []
#     l2 = []
#     l3 = []
#     l4 = []
#     l5 = []
#     l6 = []
#     l7 = []
#     for i in range(len(df['Ticker'])):
#         if i == 0:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             # print(key_metrics)
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#         else:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#     df[f'change % {v}'] = l
#     df[f'change % {v1}'] = l1
#     df[f'change % {v2}'] = l2
#     df[f'change % {v3}'] = l3
#     df[f'change % {v4}'] = l4
#     df[f'change % {v5}'] = l5
#     df[f'change % {v6}'] = l6
#     df[f'change % {v7}'] = l7
    
#     return df

# df_with_parameters = add_key_metrics(df)

In [10]:
'''
revenuePerShare < -10 = повышает шансы к лонгу
netIncomePerShare == Minus = повышает шансы к Лонгу
netIncomePerShare == Plus = повышает шансы к Шорту
bookValuePerShare > 100 = повышает шансы к Лонгу
capexPerShare > 100 = повышает шансы к Лонгу
daysPayablesOutstanding > 50 = повышает шансы к Лонгу
'''

'\nrevenuePerShare < -10 = повышает шансы к лонгу\nnetIncomePerShare == Minus = повышает шансы к Лонгу\nnetIncomePerShare == Plus = повышает шансы к Шорту\nbookValuePerShare > 100 = повышает шансы к Лонгу\ncapexPerShare > 100 = повышает шансы к Лонгу\ndaysPayablesOutstanding > 50 = повышает шансы к Лонгу\n'

add_income_statement

In [11]:
def add_income_statement(df):
    v = 'revenue'
    v1 = 'costOfRevenue'
    v2 = 'grossProfit'
    v3 = 'researchAndDevelopmentExpenses'
    v4 = 'costAndExpenses'
    v5 = 'ebitda'
    v6 = 'operatingIncome'
    v7 = 'eps'
    l = []
    l1 = []
    l2 = []
    l3 = []
    l4 = []
    l5 = []
    l6 = []
    l7 = []
    for i in range(len(df['Ticker'])):
        if i == 0:
            key_metrics = fa.income_statement(df['Ticker'][i], api_key, period='annual').T
            # print(key_metrics)
            key_metrics_change = change_percent_all_columns(key_metrics)
            l.append(key_metrics_change.iloc[0][f'change % {v}'])
            l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
            l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
            l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
            l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
            l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
            l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
            l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
        else:
            key_metrics = fa.income_statement(df['Ticker'][i], api_key, period='annual').T
            key_metrics_change = change_percent_all_columns(key_metrics)
            l.append(key_metrics_change.iloc[0][f'change % {v}'])
            l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
            l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
            l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
            l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
            l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
            l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
            l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
    df[f'change % {v}'] = l
    df[f'change % {v1}'] = l1
    df[f'change % {v2}'] = l2
    df[f'change % {v3}'] = l3
    df[f'change % {v4}'] = l4
    df[f'change % {v5}'] = l5
    df[f'change % {v6}'] = l6
    df[f'change % {v7}'] = l7
    
    return df

df_with_parameters = add_income_statement(df)

In [12]:
df_with_parameters

,Ticker,Accepted_Date,Price,Current Price,Change % price,change % debtToAssets,Side,Differense,change % revenue,change % costOfRevenue,change % grossProfit,change % researchAndDevelopmentExpenses,change % costAndExpenses,change % ebitda,change % operatingIncome,change % eps
0,RH,2022-03-30,336.119995,318.429993,-5.263002,-6.737912,Long,1.474910,31.952036,24.969815,39.974923,0.000000,18.888112,80.673025,98.600217,134.22576
1,IRBT,2022-02-15,63.400002,42.310001,-33.264984,21.932463,Long,55.197446,9.409811,33.483302,-17.893884,2.975043,21.962933,-72.339757,Minus,-78.967495
2,AOUT,2022-07-14,8.590000,10.680000,24.330618,49.753537,Long,25.422919,-10.539346,-11.058395,-9.926042,2.287096,-32.556321,Minus,Minus,Minus
3,LOW,2022-03-21,193.763824,215.869995,11.408822,14.303103,Long,2.894281,7.425472,6.945439,8.399838,0.000000,5.262039,37.060265,25.355033,55.341055
